In [53]:
import pandas as pd
pd.options.mode.chained_assignment = None

def load_df():
    df =  pd.read_excel("Local_Forecast.xlsx", sheet_name = "DEMAND")
    FC =  pd.read_excel("Local_Forecast.xlsx", sheet_name = "FORECAST")
    pareto = pd.read_excel("1. BusinessCategory.xlsx", sheet_name = "PARETO")
    return df,FC,pareto 

def calculate_avg(df):
    df['6mAVG']= df.iloc[:,2:6].mean(axis=1)
    df['3mAVG']= df.iloc[:,5:-1].mean(axis=1)
    df["Max"] = df[["6mAVG", "3mAVG"]].max(axis=1)
    Max = df[['ItemNumber','Max']]
    Max['Max'] = Max['Max']/4
    return Max

def forecast(Max,FC):
    for i in range(24):
        i = str(i)
        Max['F'+i.format(str(i))] = FC.loc[FC['Forecast_P']=="GUD_LOCAL"]["F"+i.format(str(i))].values*Max['Max']
        
    return Max

def concat(Max,df,pareto):
    Max = pd.merge(Max,df[['ItemNumber','Group']], on="ItemNumber", how='left')
    group = Max.drop(['ItemNumber','Max'], axis=1)
    group = group[[group.columns[-1]] + list(group.columns[:-1])]
    group1 = group.pivot_table(index='Group',aggfunc='sum')
    group1 = group1.reset_index()
    group1 = group1.reindex(columns=group.columns)
    
    Max = pd.merge(Max, pareto[["ItemNumber","Sls Cd3","Sls Cd4","ABC 1 Sls"]], on ='ItemNumber', how='left')
    Max = Max.round(0)
    group1 = group1.round(0)
    return Max,group1
    

### 1. Load data

In [70]:
df, FC, pareto = load_df()

### 2. Calculate Avg

In [71]:
Max = calculate_avg(df)

### 3. Forecast

In [72]:
Max = forecast(Max,FC)

### 4. Concatenate

In [73]:
Max,group1 = concat(Max,df,pareto)

In [74]:
Max.to_excel("df.xlsx", index=False)
group1.to_excel("group.xlsx", index=False)